In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [2]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Doing Prediction on the Custom Trained Model

In [3]:
import pathlib 

def load_custom_model(model_name):
  model_file = model_name

  model_dir = pathlib.Path(model_file)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [4]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/training/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [5]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/testing')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('object_detection/testing/11.jpg'),
 WindowsPath('object_detection/testing/425.jpg'),
 WindowsPath('object_detection/testing/65.jpg'),
 WindowsPath('object_detection/testing/72.jpg'),
 WindowsPath('object_detection/testing/83.jpg'),
 WindowsPath('object_detection/testing/99.jpg'),
 WindowsPath('object_detection/testing/shoo3.jpg'),
 WindowsPath('object_detection/testing/Warehouse-Gloves_1.jpg'),
 WindowsPath('object_detection/testing/warehouse-safety-float1.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0004.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0005.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0030.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0031.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0035.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0045.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0054.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0079.jpg'),
 Windo

In [6]:
model_name = 'object_detection/exported-models/my_model'
detection_model = load_custom_model(model_name)

In [7]:
print(detection_model.signatures['serving_default'].inputs)
print(detection_model.signatures['serving_default'].output_dtypes)
print(detection_model.signatures['serving_default'].output_shapes)

[<tf.Tensor 'input_tensor:0' shape=(1, None, None, 3) dtype=uint8>, <tf.Tensor 'unknown:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_0:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_1:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_2:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_3:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_4:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_5:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_6:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_7:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_8:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_9:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_10:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_11:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_12:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_13:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_14:0' shape=<unknown> dtype=resource>, <

In [8]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.7,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [9]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = image_path
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  print(output_dict)
  print(category_index)
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  return image_np

# Prediction Using Images

In [10]:
import cv2

img=cv2.imread(r'object_detection/testing/wh_dresscode_0275.jpg')
img = cv2.resize(img,(1000,640))

img=show_inference(detection_model,img)
cv2.imshow('IMG',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{'detection_boxes': array([[0.42925224, 0.46175987, 0.75948507, 0.63784236],
       [0.23067787, 0.12851365, 0.7652353 , 0.36848098],
       [0.03433855, 0.22269592, 0.1681719 , 0.36955366],
       [0.2142339 , 0.48352578, 0.31710413, 0.59516656],
       [0.42925224, 0.46175987, 0.75948507, 0.63784236],
       [0.00466077, 0.10787268, 0.64243615, 0.35949135],
       [0.20469858, 0.45480546, 0.30083928, 0.56635755],
       [0.3207005 , 0.21677299, 0.75014985, 0.38812506],
       [0.3629148 , 0.3846604 , 0.8080461 , 0.6218663 ],
       [0.        , 0.23126091, 0.12697552, 0.38872582],
       [0.40447792, 0.49898583, 0.8277837 , 0.7031756 ],
       [0.22763208, 0.47733614, 0.30836278, 0.54283625],
       [0.26641372, 0.13005245, 0.797568  , 0.36301276],
       [0.20683494, 0.5094188 , 0.28724363, 0.60152715],
       [0.47406566, 0.44414744, 0.7687389 , 0.56590235],
       [0.49766204, 0.127702  , 0.76854813, 0.4078138 ],
       [0.42925224, 0.46175987, 0.75948507, 0.63784236],
       [0.2

# Prediction using Video

In [ ]:
import cv2

video=cv2.VideoCapture(r'F:\ware\vid1.mp4')


while(True):
    ret,img=video.read()
    img=show_inference(detection_model,img)
    cv2.imshow('LIVE',img)
    k=cv2.waitKey(1)
    
    if(k==27):
        break
cv2.destroyAllWindows()